## Generation of a dataset of text from Mistral7B model (base model) for classifier training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Creation of dataset for clf training


In [ ]:
!pip install -U simpletransformers
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops gradio sentencepiece

In [ ]:
import torch
import simpletransformers
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, wandb, platform, gradio, warnings
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login
import json
from tqdm import tqdm

In [ ]:
# data and models paths
dir_root = './drive/MyDrive/DL-ENS'
dir_data = f'{dir_root}/dataset'
list_to_generate_path = f'{dir_data}/story_prompts_2.txt'

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

In [ ]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
   model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

In [ ]:
# function to tokenize the input in the expected form of the prompt
def tokenize(tokenizer, text):
  return tokenizer(f"<s>[INST]This are the first lines of a work of fiction. Continue it. {text} [/INST]", return_tensors = "pt", add_special_tokens = False)

In [ ]:
# function for texts generation
def generate_texts(model, tokenizer, texts):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  generated_texts = []
  for input in tqdm(texts):
    tokens = tokenize(tokenizer, input)
    model_inputs = tokens.to(device)
    generated_ids = model.generate(**model_inputs, max_new_tokens=500, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    generated_texts.extend(decoded)
    del model_inputs
    del decoded
    del generated_ids
  return generated_texts

In [ ]:
texts = []
with open(list_to_generate_path, 'r+', encoding='utf-8') as fd:
  texts = fd.readlines()
texts = [text[:-1] for text in texts]

In [ ]:
generated_texts = generate_texts(model, tokenizer, texts)

In [ ]:
# function to save the results
def save_generated_texts(texts):
  dict_generated_texts = {'texts': []}

  for i in range(len(texts)):
    dict_generated_texts['texts'].append(texts[i])

  with open(dir_data + f"/dataset_mistral7B_gen_texts.json", 'w+') as fd:
    json.dump(dict_generated_texts, fd)